In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow import keras 

from tensorflow.keras import Model, metrics, backend

from tensorflow.keras.layers import LSTM, Input, Embedding, Dense, Dropout

from tensorflow.keras.losses import CategoricalCrossentropy

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import math
from datetime import datetime as dt
import os
import matplotlib.pyplot as plt

# Load and transform values

In [ ]:
# RNN inputs -- Transform y_label and perform OneHotEnconding on failure type

rnn_inputs_event = np.load('event_set.npy')
profile = pd.read_pickle('profile.pickle')
profile = profile.drop(columns='unit_ID')

scaler_time = MinMaxScaler()
d1 = rnn_inputs_event.shape[0]
d2 = rnn_inputs_event.shape[1]

num_class = np.unique(rnn_inputs_event[:,:,0]).shape[0]
rnn_input_onehot = np.array(pd.get_dummies(pd.DataFrame(rnn_inputs_event[:,:,0].reshape(-1,1).astype(str)))).reshape(d1,d2,num_class)
rnn_inputs_event = np.concatenate([rnn_input_onehot, rnn_inputs_event[:,:,1:]], axis=2)

In [ ]:
# Perform normalization and dummies for static features

scaler_lengte = MinMaxScaler()
scaler_age = MinMaxScaler()
scaler_appendage = MinMaxScaler()
scaler_water_hammer = MinMaxScaler()

# test diameters -> categorical or numerical
scaler_inw = MinMaxScaler()
scaler_nom = MinMaxScaler()

profile['LENGTE_GIS'] = scaler_lengte.fit_transform(np.array(profile['LENGTE_GIS']).reshape(-1, 1))

profile['Age'] = scaler_age.fit_transform(np.array(profile['Age']).reshape(-1, 1))

profile['Aansluiting'] = scaler_appendage.fit_transform(np.array(profile['Aansluiting']).reshape(-1, 1))

profile['distance'] = scaler_water_hammer.fit_transform(np.array(profile['distance']).reshape(-1, 1))

profile['INWENDIGE_'] = scaler_inw.fit_transform(np.array(profile['INWENDIGE_']).reshape(-1, 1))
profile['NOMINALE_D'] = scaler_nom.fit_transform(np.array(profile['NOMINALE_D']).reshape(-1, 1))

profile = pd.get_dummies(profile, prefix_sep='_').values

In [ ]:
def train_val(rnn_inputs_event, profile, p=[0.25, 0.75]):
    global x_train_event, y_train_event, x_val_event, y_val_event, profile_train, profile_val
    
    # Randomization of data file

    boolean_train = np.random.choice(a=[False, True], p=p, size=(profile.shape[0],))
    boolean_val = b = np.invert(boolean_train)

    x_train_event = rnn_inputs_event[:,:-1,:]
    y_train_event = rnn_inputs_event[:,:-1,-1]
    
    x_train_event = rnn_inputs_event[:,:-1,:][boolean_train]
    y_train_event = rnn_inputs_event[:,-1:,-1][boolean_train]
    x_val_event = rnn_inputs_event[:,:-1,:][boolean_val]
    y_val_event = rnn_inputs_event[:,-1:,-1][boolean_val]
    profile_train = profile[boolean_train]
    profile_val = profile[boolean_val]
    
    
    
    return x_train_event, y_train_event, x_val_event, y_val_event, profile_train, profile_val

train_val(rnn_inputs_event, profile, [0.25, 0.75])

print('Shape x_train: ' + str(x_train_event.shape))
print('Shape x_val: ' + str(x_val_event.shape))
print()
print('Shape y_train: ' + str(y_train_event.shape))
print('Shape y_val: ' + str(y_val_event.shape))
print()
print('Shape profile_train: ' + str(profile_train.shape))
print('Shape profile_val: ' + str(profile_val.shape))


In [ ]:
plt.boxplot(y_train_event)

# RNN LSTM model

In [ ]:
TYPE = 'joint'  # Set type of model

num_steps = x_train_event.shape[1]
event_size =  x_train_event.shape[2]
profile_size = profile_train.shape[1]

main_input = Input(shape=(num_steps,event_size), dtype='float32', name='main_input')
lstm_out = LSTM(32)(main_input)

auxiliary_input = Input(shape=(profile_size,), name='aux_input')

if TYPE=='joint':
    x = keras.layers.concatenate([lstm_out, auxiliary_input])
elif TYPE=='event':
    x = lstm_out
elif TYPE=='profile':
    x = auxiliary_input

x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

time_stamp = Dense(1, activation='softplus', name='time_stamp')(x)

if TYPE=='joint':
    model = Model(inputs=[main_input, auxiliary_input], outputs=[time_stamp])
elif TYPE=='event':
    model = Model(inputs=[main_input], outputs=[time_stamp])
elif TYPE=='profile':
    model = Model(inputs=[auxiliary_input], outputs=[time_stamp])


model.compile(optimizer='adam',
              loss='mse',
              metrics=['mae'])

In [ ]:
# And trained it via:
model.fit({'main_input': x_train_event, 'aux_input': profile_train},
          {'time_stamp': y_train_event},
          epochs=30,
          batch_size=64,
          verbose=2,
          validation_data=({'main_input': x_val_event, 'aux_input': profile_val},
          {'time_stamp': y_val_event}))

In [ ]:
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test and validation data')
results_train = model.evaluate({'main_input': x_train_event, 'aux_input': profile_train},
          {'time_stamp': y_train_event}, batch_size=32,
          verbose=0)

results_val = model.evaluate({'main_input': x_val_event, 'aux_input': profile_val},
          {'time_stamp': y_val_event}, batch_size=32,
          verbose=0)

print('Loss value on test set: ' + str(results_train[0]) + ' | Accuracy (MAE) on test set: ' + str(results_train[1]))
print('Loss value on validation set: ' + str(results_val[0]) + ' | Accuracy (MAE) on validation set: ' + str(results_val[1]))

In [ ]:
results = model.predict({'main_input': x_val_event, 'aux_input': profile_val})